# 01. b. Machine Learning in Cloud
One does not simply become a data scientist...

### Author: Daniel Kurniadi, MLDA Executive in Academic Division

<img src="images/AWS_Banner.jpg"/>

### Abstract:

The purpose of this research is to try a machine learning approach for predicting Diabetes diagnosis for medical application by looking at medical checkup result. The research contains: Data exploration, feature engineering, choosing appropriate scoring metric, cross algorithms, cross validation, tuning the algorithms,analysis of feature importance, analysis of residuals and performance evaluation. The used dataset is from years 1994.


### Introduction:

Diabetes patient records were obtained from two sources: an automatic electronic recording device and paper records. The automatic device had an internal clock to timestamp events, whereas the paper records only provided "logical time" slots (breakfast, lunch, dinner, bedtime). For paper records, fixed times were assigned to breakfast (08:00), lunch (12:00), dinner (18:00), and bedtime (22:00). Thus paper records have fictitious uniform recording times whereas electronic records have more realistic time stamps.

## Workshop Outline
- <u>01.a.Introduction into Data Analytics (this workshop)
- [01.b.Machine Learning in Cloud](#02.ML_in_Cloud.ipynb)

## Contents

1. [AWS Setup Configuration](#AWS-Setup-Configuration)
1. [Meet and Greet Data](#Data)
1. [Train](#Train)
1. [Compile](#Compile)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Relative cost of errors](#Relative-cost-of-errors)
1. [Extensions](#Extensions)

## Case Study: Diabetes Prediction
Working through machine learning problems from end-to-end is critically important. You can read about machine learning. You can also try out small one-off recipes. But applied machine learning will not come alive for you until you work through a dataset from beginning to end.

Working through a project forces you to think about how the model will be used, to challenge your assumptions and to get good at all parts of a project, not just your favorite parts. The best way to practice predictive modeling machine learning projects is to use standardized datasets from the UCI Machine Learning Repository or hone your skill on Kaggle data science competition. Once you have a practice dataset and a bunch of recipes, how do you put it all together and work through the problem end-to-end?

### Data

The diabetes data set was originated from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes) or [Kaggle: Diabetes data](https://www.kaggle.com/edubrq/diabetes). This dataset describes the medical records for Pima Indians and whether or not each patient will have an onset of diabetes within five years. 

**Data Dictionary**
* `Pregnancies`: Number of times pregnant 
* `Glucose`: Plasma glucose concentration a 2 hours in an oral glucose tolerance test 
* `BloodPressure`: Diastolic blood pressure (mm Hg) 
* `SkinThickness`: Triceps skin fold thickness (mm) 
* `Insulin`: 2-Hour serum insulin (mu U/ml) 
* `BMI`: Body mass index (weight in kg/(height in m)^2) 
* `DiabetesPedigreeFunction`: Diabetes pedigree function 
* `Age`: Age (years) 
* `Outcome`: Class variable (0 or 1) 

## AWS Setup Configuration

This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).


In [ ]:
bucket = 'aws-lta-workshop2020'
prefix = 'sagemaker/xgboost-ml-in-cloud'

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

### Packages and Libraries

In [ ]:
# standard libraries
import io, os, sys
import time, json
from time import strftime, gmtime

In [ ]:
import numpy  as np                                    # import numpy for number handling
import pandas as pd                                    # import pandas for data wrangling
import matplotlib.pyplot as plt                        # import matplotlib for ploting
%matplotlib inline

import sklearn                                         # package for predictive modeling
from sklearn import metrics                            # metrics package to evaluate the models

In [ ]:
import warnings

# silent warning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# AWS sagemaker API
import sagemaker
from sagemaker.predictor import csv_serializer

## 1. Meet and Greet Data

### Data Structure 
After reading in the data, we first do some simple exploration, check available columns, data structure, and data summary.

In [ ]:
PATH_TO_DATA = 'data/Diabetes/diabetes.csv'
data = pd.read_csv(PATH_TO_DATA)

In [ ]:
data.head(5) # take a look at the first 5 rows

In [ ]:
print("Rows, Columns: ", data.shape)

## 2. Modellling and Model Evaluation

In the [previous workshop](#Week1_Diabetes_Prediction), we have seen many algorithms that we can use to make binary classification such as Random Forest and Gradient Boosting. Let's attempt to model this problem using gradient boosted trees.

Amazon SageMaker provides an <u>XGBoost</u> container that we can use to train in a managed, distributed setting, and then host as a real-time prediction endpoint.  <u>XGBoost</u> uses gradient boosted trees which naturally account for non-linear relationships between features and the target variable, as well as accommodating complex interactions between features.

Amazon SageMaker <u>XGBoost</u> can train on data in either a CSV or LibSVM format.  For this example, we'll stick with CSV.  It should:
- Have the predictor variable in the first column
- Not have a header row

And now let's split the data into training, validation, and test sets. This will help prevent us from overfitting the model, and allow us to test the models accuracy on data it hasn't already seen.

### 2.1 Train- Validation-Test splits

In [ ]:
from sklearn.model_selection import train_test_split

# Split training set into train and validation
X = data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
          'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
Y = data['Outcome']

# Split the dataset (X and Y) into training and testing sets (95%:5%)
msk = np.random.rand(len(df)) < 0.95

X_train, X_test = X[msk], X[~msk]
Y_train, Y_test = Y[msk], Y[~msk]

# Split the training set further to training and validation sets (76%:19%)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)

As mentioned, we will need to have the predictor/target variable in the first column in our dataset.

Here, the code below will concatenate back our predictor variable and place it as the first column in our dataset.

In [ ]:
train_data = pd.concat([Y_train, X_train], axis=1)
validation_data = pd.concat([Y_val, X_val], axis=1)

We save the dataset that we have split into training and validation set into a *csv* format

In [ ]:
# save training and test set to file
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

Now we'll upload these csv files to S3 bucket

In [ ]:
# move training set file to s3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv'))\
    .upload_file('train.csv')
# move test set file to s3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv'))\
    .upload_file('validation.csv')

Moving onto training, first we'll need to specify the locations of the XGBoost algorithm containers.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')

Then, because we're training with the CSV file format, we'll create a pointer to the files in S3.
We call this pointer `s3_input`

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'\
                                    .format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'\
                                         .format(bucket, prefix), content_type='csv')

### 2.2 Model Fitting: XGBoost

Now, we can specify a few parameters like what type of training instances we'd like to use and how many, as well as our XGBoost hyperparameters.  A few key hyperparameters are:
- `max_depth` controls how deep each tree within the algorithm can be built.  Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting.  There is typically some trade-off in model performance that needs to be explored between a large number of shallow trees and a smaller number of deeper trees.
- `subsample` controls sampling of the training data.  This technique can help reduce overfitting, but setting it too low can also starve the model of data.
- `num_round` controls the number of boosting rounds.  This is essentially the subsequent models that are trained using the residuals of previous iterations.  Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.
- `eta` controls how aggressive each round of boosting is.  Larger values lead to more conservative boosting.
- `gamma` controls how aggressively trees are grown.  Larger values lead to more conservative models.

More detail on XGBoost's hyperparmeters can be found on their documentation [page](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html).

In [ ]:
sess = sagemaker.Session()

# STEP 1: define a sagemaker estimator object with specified container
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.c4.4xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

# STEP 2: tune and specify hyperparameters related to your model
xgb.set_hyperparameters(num_round=15,                 # epochs
                        objective='binary:logistic',  # use negative-log-likelihood loss function
                        max_depth=6,                  # maximum tree depth
                        eta=0.1,                      # feature regularization term
                        gamma=4,                      # gain threshold for tree branching
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0)

# STEP 3: train and evaluate model in cloud
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

## 3. Model Deployment: Sagemaker Neo

### 3.1 Compile
[Amazon SageMaker Neo](https://aws.amazon.com/sagemaker/neo/) optimizes models to run up to twice as fast, with no loss in accuracy. When calling `compile_model()` function, we specify the target instance family (m4) as well as the S3 bucket to which the compiled model would be stored.

In [ ]:
compiled_model = xgb

# STEP 4: Compile model using sagemaker neo
try:
    xgb.create_model()._neo_image_account(boto3.Session().region_name)
except:
    print('Neo is not currently supported in', boto3.Session().region_name)
else:
    output_path = '/'.join(xgb.output_path.split('/')[:-1])
    compiled_model = xgb.compile_model(target_instance_family='ml_m4', 
                                   input_shape={'data':[1, 69]},
                                   role=role,
                                   framework='xgboost',
                                   framework_version='0.7',
                                   output_path=output_path)
    compiled_model.name = 'deployed-xgboost-diabetes'
    compiled_model.image = get_image_uri(sess.boto_region_name, 'xgboost-neo', repo_version='latest')

### 3.2 Host

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [ ]:
# STEP 5: Deploy compiled model in cloud
xgb_predictor = compiled_model.deploy(initial_instance_count = 1, instance_type = 'ml.c4.4xlarge')

### 3.3 Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request.  But first, we'll need to setup serializers and deserializers for passing our `test_data` NumPy arrays to the model behind the endpoint.

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

Now, we'll use a simple function to:

1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batchs to CSV string payloads
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
def predict(data, rows=20):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(validation_data.values[:, 1:])
y_pred = predictions > 0.5

#### Confusion Matrix

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f'
    thresh = cm.max()/2.
    
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
        horizontalalignment = "center",
        color = "white" if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(Y_val, y_pred)

# Display confusion matrix
plt.figure(figsize=(8,8))
plot_confusion_matrix(confusion_matrix, classes=["Negative", "Positive"])

#### ROC AUC

In [ ]:
fpr, tpr, _ = metrics.roc_curve(Y_val,  predictions)
auc = metrics.roc_auc_score(Y_val, predictions)

# plot ROC AUC curve
plt.figure(figsize=(8,8))
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

### 4. Cleaning Up

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)